In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import os

In [2]:
from nltk.corpus import stopwords 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from nltk.tokenize import TweetTokenizer, word_tokenize, sent_tokenize

In [3]:
raw_data = pd.read_csv('ruu_cipta_kerja_20102020_terbaru.csv')
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [4]:
raw_data.head()

,UserName,Handle,Timestamp,Text,Emojis,Comments,Likes,Retweets
0,Fathiyyah Rahma,@RahmaFathiyyah,2020-10-20T04:57:59.000Z,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,NaN,NaN,NaN,NaN
1,Dwi Hendriani,@Hendria04991579,2020-10-20T04:51:39.000Z,Jurus kilat ala sistem kebut skripsi layaknya ...,NaN,NaN,NaN,NaN
2,Dwiina,@Dwiina7,2020-10-20T04:51:27.000Z,Jurus kilat ala sistem kebut skripsi layaknya ...,NaN,NaN,NaN,NaN
3,anis,@anis34216465,2020-10-20T04:51:23.000Z,Jurus kilat ala sistem kebut skripsi layaknya ...,NaN,NaN,NaN,NaN
4,Ayuniie Dwi,@Ayuniie12,2020-10-20T04:51:16.000Z,Jurus kilat ala sistem kebut skripsi layaknya ...,NaN,NaN,NaN,NaN


In [5]:
df = pd.DataFrame(raw_data[['UserName', 'Text']])
df

,UserName,Text
0,Fathiyyah Rahma,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...
1,Dwi Hendriani,Jurus kilat ala sistem kebut skripsi layaknya ...
2,Dwiina,Jurus kilat ala sistem kebut skripsi layaknya ...
3,anis,Jurus kilat ala sistem kebut skripsi layaknya ...
4,Ayuniie Dwi,Jurus kilat ala sistem kebut skripsi layaknya ...
...,...,...
15167,Titikrindu,Membalas \n@ComikSnTOLAK RUU CIPTA KERJA\n\n#M...
15168,Nananana,Membalas \n@pearapples_TOLAK RUU CIPTA KERJA\n...
15169,squitwooot,Membalas \n@ssunlighhtTOLAK RUU CIPTA KERJA\n\...
15170,Fauziah Rachmawati,"Jadi.. sejak digagas oleh Menko Perekonomian, ..."


In [6]:
df.head()

,UserName,Text
0,Fathiyyah Rahma,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...
1,Dwi Hendriani,Jurus kilat ala sistem kebut skripsi layaknya ...
2,Dwiina,Jurus kilat ala sistem kebut skripsi layaknya ...
3,anis,Jurus kilat ala sistem kebut skripsi layaknya ...
4,Ayuniie Dwi,Jurus kilat ala sistem kebut skripsi layaknya ...


In [7]:
# remove symbol @ and #
def remove_pattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for i in r:
        tweet = re.sub(i, '', tweet)
    return tweet

df['RemoveUser'] = np.vectorize(remove_pattern)(df['Text'], "(@\\w*)")
df['RemoveSymbol'] = df["RemoveUser"].apply(lambda x: np.vectorize(remove_pattern)(x, "(#\\w*)"))

In [8]:
df.head()

,UserName,Text,RemoveUser,RemoveSymbol
0,Fathiyyah Rahma,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...
1,Dwi Hendriani,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...
2,Dwiina,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...
3,anis,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...
4,Ayuniie Dwi,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...


In [9]:
def concate_duplicate(tweet):
    term = "a" + r"{3}"
    rep = re.sub(term, " 3", tweet)
    term = "i" + r"{3}"
    rep = re.sub(term, " 3", rep)
    term = "u" + r"{3}"
    rep = re.sub(term, " 3", rep)
    term = "e" + r"{3}"
    rep = re.sub(term, " 3", rep)
    term = "o" + r"{3}"
    rep = re.sub(term, " 3", rep)
    
    term = "c" + r"{3}"
    rep = re.sub(term, " 3", rep)
    term = "k" + r"{3}"
    rep = re.sub(term, " 3", rep)
    term = "w" + r"{3}"
    rep = re.sub(term, " 3", rep)
    term = "h" + r"{3}"
    rep = re.sub(term, " 3", rep)
    
    return rep

df['RemoveDuplicateChar'] = df["RemoveSymbol"].apply(lambda x: concate_duplicate(x))

In [10]:
# normalisasi singkatan 3 karakter
def get_dictionary():
#     dir_path = os.path.dirname(os.path.realpath(__file__))
    dir_path = os.path.abspath("kata3karakter.csv")
    df = pd.read_csv(dir_path, sep=';')
    dictlist = []
    for row in df.values:
        dictlist.append([row[0], row[1]])
    return dictlist

kamus = get_dictionary()

In [11]:
# normalisasi kata
def get_dictionary_norm():
#     dir_path = os.path.dirname(os.path.realpath(__file__))
    dir_path = os.path.abspath("katanormal.csv")

    df = pd.read_csv(dir_path, sep=';')
    dictlist = []
    for row in df.values:
        dictlist.append([row[0], row[1]])
    return dictlist

kamus_normal = get_dictionary_norm()

In [12]:
def remove_url(text):
    # Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    text = re.sub('[\n]+', ' ', text)
    
    # Remove single char
    text = re.sub(r"\b[a-zA-Z]\b", "", text)

    # remove all url
    text = re.sub(r" ?(f|ht)(tp)(s?)(://)(.*)[.|/](.*)", "", text)

    # remove email
    text = re.sub(r"[\w]+@[\w]+\.[c][o][m]", "", text)
    # text = re.sub(r"[\w]+@[\w]+\.[c][o]", "", text)
    # text = re.sub(r"[\w]+@[\w]+\.[o][r][g]", "", text)
    # remove text twit
    text = re.sub(r'((pic\.[^\s]+)|(twitter))', '', text)
    # remove mentions, hashtag and web
    text = re.sub(r"(?:\@|#|http?\://)\S+", "", text)
    # remove url
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub(r'((https?://[^\s]+))', '', text)
    text = re.sub(r"(pic[^\s]+)|[\w]+\.[c][o][m]", "", text)
    # replace non ascii
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)


    return text

def remove_symbol(tweet):
    tweet = remove_url(tweet)

    # get only alfabet
    pattern = re.compile(r'\b[^\d\W]+\b')
    newwords = []
    for word in pattern.findall(tweet):
        # case folding
        word = word.lower()

        for row in kamus:
            key = row[0]
            value = row[1]
            if word == key:
                word = value
                break

        for row in kamus_normal:
            key = row[0]
            value = row[1]
            if word == key:
                word = value
                break

        word = word.replace("xyz", "")
        newwords.append(word)
    return " ".join(newwords)

In [13]:
# remove emoji
def remove_emojis(data):
    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

df['RemoveEmoji'] = df['RemoveDuplicateChar'].apply(lambda x: remove_emojis(remove_symbol(x)))
df

,UserName,Text,RemoveUser,RemoveSymbol,RemoveDuplicateChar,RemoveEmoji
0,Fathiyyah Rahma,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,ketua dewan perwakilan rakyat draft rancangan ...
1,Dwi Hendriani,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,jurus kilat ala sistem kebut skripsi layaknya ...
2,Dwiina,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,jurus kilat ala sistem kebut skripsi layaknya ...
3,anis,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,jurus kilat ala sistem kebut skripsi layaknya ...
4,Ayuniie Dwi,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,jurus kilat ala sistem kebut skripsi layaknya ...
...,...,...,...,...,...,...
15167,Titikrindu,Membalas \n@ComikSnTOLAK RUU CIPTA KERJA\n\n#M...,Membalas \n RUU CIPTA KERJA\n\n#MahasiswaBerge...,Membalas \n RUU CIPTA KERJA\n\n \n \n \n \n \n,Membalas \n RUU CIPTA KERJA\n\n \n \n \n \n \n,membalas rancangan undang undang cipta kerja
15168,Nananana,Membalas \n@pearapples_TOLAK RUU CIPTA KERJA\n...,Membalas \n RUU CIPTA KERJA\n\n#MosiTidakPerca...,Membalas \n RUU CIPTA KERJA\n\n \n\n\n\n\n\n\n \n,Membalas \n RUU CIPTA KERJA\n\n \n\n\n\n\n\n\n \n,membalas rancangan undang undang cipta kerja
15169,squitwooot,Membalas \n@ssunlighhtTOLAK RUU CIPTA KERJA\n\...,Membalas \n RUU CIPTA KERJA\n\n#StopRugikanRak...,Membalas \n RUU CIPTA KERJA\n\n\n \n\n\n\n \n ...,Membalas \n RUU CIPTA KERJA\n\n\n \n\n\n\n \n ...,membalas rancangan undang undang cipta kerja
15170,Fauziah Rachmawati,"Jadi.. sejak digagas oleh Menko Perekonomian, ...","Jadi.. sejak digagas oleh Menko Perekonomian, ...","Jadi.. sejak digagas oleh Menko Perekonomian, ...","Jadi.. sejak digagas oleh Menko Perekonomian, ...",jadi sejak digagas oleh menko perekonomian ran...


In [14]:
df.drop_duplicates(subset = "RemoveSymbol", keep = 'first', inplace = True)
df.dropna()
df.head()
df

,UserName,Text,RemoveUser,RemoveSymbol,RemoveDuplicateChar,RemoveEmoji
0,Fathiyyah Rahma,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,Ketua DPR: Draft RUU Cipta Kerja Hanya Dicek A...,ketua dewan perwakilan rakyat draft rancangan ...
1,Dwi Hendriani,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,Jurus kilat ala sistem kebut skripsi layaknya ...,jurus kilat ala sistem kebut skripsi layaknya ...
6,anis,Sudah jatuh tertimpa tangga. Bebas Covid-19 ja...,Sudah jatuh tertimpa tangga. Bebas Covid-19 ja...,Sudah jatuh tertimpa tangga. Bebas Covid-19 ja...,Sudah jatuh tertimpa tangga. Bebas Covid-19 ja...,sudah jatuh tertimpa tangga bebas covid jauh p...
8,SINDOnews,Tokoh KAMI Ahmad Yani Akui Hendak Diangkut Pol...,Tokoh KAMI Ahmad Yani Akui Hendak Diangkut Pol...,Tokoh KAMI Ahmad Yani Akui Hendak Diangkut Pol...,Tokoh KAMI Ahmad Yani Akui Hendak Diangkut Pol...,tokoh kami ahmad yani akui hendak diangkut pol...
9,Eko Firman,Mahasiswa cerdas . Pelajaran i dulu RUU CIPTA ...,Mahasiswa cerdas . Pelajaran i dulu RUU CIPTA ...,Mahasiswa cerdas . Pelajaran i dulu RUU CIPTA ...,Mahasiswa cerdas . Pelajaran i dulu RUU CIPTA ...,mahasiswa cerdas pelajaran dulu rancangan unda...
...,...,...,...,...,...,...
15160,friska`,"Membalas \n@ocodabonitolong takedown dulu ya, ...","Membalas \n takedown dulu ya, respect!\n\nTOLA...","Membalas \n takedown dulu ya, respect!\n\nTOLA...","Membalas \n takedown dulu ya, respect!\n\nTOLA...",membalas takedown dulu ya kehormatan tolak ran...
15165,efri deplin,RUU Cipta Kerja adalah RUU inisiatif Pemerinta...,RUU Cipta Kerja adalah RUU inisiatif Pemerinta...,RUU Cipta Kerja adalah RUU inisiatif Pemerinta...,RUU Cipta Kerja adalah RUU inisiatif Pemerinta...,rancangan undang undang cipta kerja adalah ran...
15166,Sumiyati Sapriasih,Hi ... tweeps apa kabar ?\nPagi ini saya akan ...,Hi ... tweeps apa kabar ?\nPagi ini saya akan ...,Hi ... tweeps apa kabar ?\nPagi ini saya akan ...,Hi ... tweeps apa kabar ?\nPagi ini saya akan ...,tweeps apa kabar pagi ini saya akan mengulas ...
15167,Titikrindu,Membalas \n@ComikSnTOLAK RUU CIPTA KERJA\n\n#M...,Membalas \n RUU CIPTA KERJA\n\n#MahasiswaBerge...,Membalas \n RUU CIPTA KERJA\n\n \n \n \n \n \n,Membalas \n RUU CIPTA KERJA\n\n \n \n \n \n \n,membalas rancangan undang undang cipta kerja


In [15]:
stopwords_indonesia = stopwords.words('indonesian')
stop_factory = StopWordRemoverFactory().get_stop_words()
dictionary = ArrayDictionary(stop_factory)
swr = StopWordRemover(dictionary)

def clean_tweets(tweet):
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(swr.remove(tweet))

    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            tweets_clean.append(word)

    stem_word = stemmer.stem(" ".join(tweets_clean)) # stemming word
    return stem_word


df['TweetClean'] = df['RemoveEmoji'].apply(lambda x: clean_tweets(x))

In [16]:
df.to_csv("result.csv")

In [17]:
df['TweetClean'].to_csv("result_clean.csv")

In [18]:
load_data = pd.read_csv("result_clean.csv")
load_data.dropna()
print(load_data)

       Unnamed: 0                                         TweetClean
0               0  ketua dewan wakil rakyat draft rancang undang ...
1               1  jurus kilat ala sistem kebut skripsi layak mah...
2               6  jatuh timpa tangga bebas covid panggang api pa...
3               8  tokoh ahmad yani aku angkut polisi ahmad yani ...
4               9  mahasiswa cerdas ajar rancang undang undang ci...
...           ...                                                ...
10241       15160  balas takedown hormat tolak rancang undang und...
10242       15165  rancang undang undang cipta kerja rancang unda...
10243       15166  tweeps kabar pagi ulas rancang undang undang c...
10244       15167            balas rancang undang undang cipta kerja
10245       15170  gagas menko ekonomi rancang undang undang cipt...

[10246 rows x 2 columns]


In [19]:
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]



In [20]:
df = pd.DataFrame(load_data[['TweetClean']])
df

,TweetClean
0,ketua dewan wakil rakyat draft rancang undang ...
1,jurus kilat ala sistem kebut skripsi layak mah...
2,jatuh timpa tangga bebas covid panggang api pa...
3,tokoh ahmad yani aku angkut polisi ahmad yani ...
4,mahasiswa cerdas ajar rancang undang undang ci...
...,...
10241,balas takedown hormat tolak rancang undang und...
10242,rancang undang undang cipta kerja rancang unda...
10243,tweeps kabar pagi ulas rancang undang undang c...
10244,balas rancang undang undang cipta kerja


In [21]:
        countWords = []        
        allwords = []
        for text in load_data['TweetClean'].values:
            if isinstance(text, str):
                if text != "":
                    t = text.split(" ")
                    for alc in t:
                        allwords.append(alc)
                    countWords.append(text)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [23]:
#untuk menghitung jumlah kata yang telah di steming
cv = CountVectorizer()
term_fit = cv.fit(allwords)

print (len(term_fit.vocabulary_))

12732


In [24]:
print (term_fit.get_feature_names())

 'sisa', 'sisdiknas', 'sisi', 'sisip', 'sisir', 'sisss', 'sist', 'sistem', 'sistematis', 'sisters', 'sistim', 'siswa', 'sita', 'site', 'sitenya', 'siti', 'sitindaon', 'situ', 'situasi', 'situbondo', 'situkangmarahj', 'situs', 'sitweet', 'siuman', 'siur', 'sivil', 'sjmlh', 'sjsn', 'sjumlah', 'sjw', 'sk', 'skala', 'skali', 'skandal', 'skarang', 'skck', 'skdr', 'skedar', 'skelompom', 'skenario', 'skg', 'skill', 'skillnya', 'skincare', 'skip', 'skor', 'skr', 'skrang', 'skrg', 'skripshit', 'skripsi', 'skrng', 'sks', 'skuy', 'slah', 'slain', 'slama', 'slamet', 'slanjutnya', 'slank', 'slavery', 'sleep', 'sleman', 'slewengkan', 'slide', 'slmt', 'slof', 'slogan', 'slot', 'slow', 'slsai', 'slt', 'slur', 'sma', 'smakin', 'smart', 'smartphone', 'smd', 'smesta', 'smg', 'smga', 'smi', 'smk', 'smlem', 'smoga', 'smp', 'smpai', 'smpaikan', 'smpe', 'smrc', 'sms', 'smth', 'smua', 'smuanya', 'smw', 'smwa', 'snap', 'sndal', 'sndiri', 'sndri', 'sngt', 'snip', 'sniper', 'so', 'soal', 'soale', 'soalnyanyg', '

In [25]:
list = term_fit.get_feature_names()
norm_job = pd.DataFrame(term_fit.get_feature_names(), columns=["colummn"])
norm_job.to_csv('norm_job.csv', index=False)